# **Install, import packages and authenticating**

In [43]:
!pip install git+https://github.com/spaceml-org/ml4floods#egg=ml4floods
!pip install pygeos

  Cloning https://github.com/spaceml-org/ml4floods to /tmp/pip-install-gnzqn67_/ml4floods_184ac9f52225497486aedf17c91b9eee
  Running command git clone --filter=blob:none --quiet https://github.com/spaceml-org/ml4floods /tmp/pip-install-gnzqn67_/ml4floods_184ac9f52225497486aedf17c91b9eee
  Resolved https://github.com/spaceml-org/ml4floods to commit c49a676b8b17c0a170bb2c071abf206d563f713e
  Preparing metadata (setup.py) ... done


In [55]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import pygeos
gpd.options.use_pygeos = True

import ee
from ml4floods.data import ee_download
from datetime import datetime, timedelta, timezone
import geemap.eefolium as geemap
import folium

import matplotlib.pyplot as plt

In [45]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1ARtbsJqyKOVkGl1uWaYK3Ckfd4zy_bJudyyrAIf5LPn8k_zqusE_kGux4JE



Successfully saved authorization token.


# **Getting the buildings affected by the floods in the government supplied AOI.**

In [77]:
# Returns a geodata with the buildings affected in the government supplied AOI.

def get_intersection(aoi_footprintPath: str, aoi_mapPath: str ):
    
    # Reading the files in.
    aoi_footprint = gpd.read_file(aoi_footprintPath)
    aoi_map = gpd.read_file(aoi_mapPath)
    
    # All classes will be plotted later
    aoi_floodTrace= aoi_map.groupby('class').get_group('flood-trace')
    aoi_preFlood= aoi_map.groupby('class').get_group('water-pre-flood')
    aoi_postFlood= aoi_map.groupby('class').get_group('water-post-flood')    
    
    # Saving the index as it will be used in the intersection
    aoi_footprint['footprint_index'] = aoi_footprint.index
    
    # Finding the affected buildings in the government supplied AOI. 
    aoi_intersect = gpd.overlay(aoi_floodTrace, aoi_footprint, how='intersection')
    
    # Using the index of the intersections to get the full original shape of the buildings
    # If this isn't done, only some of footprint of the building that is intersecting with the flood 
    # will be stored. 
    intersection = aoi_footprint.loc[aoi_intersect['footprint_index']]
    intersection = intersection.drop(columns= 'footprint_index')
    
    # Including the index as it's own column for plotting purposes
    intersection['index']= intersection.index
    
    # Number of affected buildings.
    length = len(intersection)
    print(f'There are {length} buildings affected in this AOI')
    
    # Saving the file. 
    intersection.to_file("PACE/data/aoi_intersect.geojson", driver='GeoJSON')
    
    # We now want to get the proportion of the building that is affected 
    # by the flood. First we convert to UTM as the `area` function doesn't 
    # like wgs crs. 
    aoi_intersect_new = aoi_intersect['geometry'].to_crs('EPSG:32733')
    aoi_intersect_new_area = aoi_intersect_new.area

    intersection_new = intersection['geometry'].to_crs('EPSG:32733')
    intersection_new_area = intersection_new.reset_index()['geometry'].area
    
    # we use the `set_axis` function to update the index to match the 'intersection' geodatafram
    floodCover = (aoi_intersect_new_area/intersection_new_area).set_axis(intersection.index)
    intersection['flood_cover'] = floodCover
    
    
    ##-----------------Visualisation with geemap-----------------
    date_event = datetime.strptime("2022-06-01","%Y-%m-%d").replace(tzinfo=timezone.utc)

    # Code modified from Gonzalo's tutorial 
    def generate_polygon(bbox):
        """
        Generates a list of coordinates: [[x1,y1],[x2,y2],[x3,y3],[x4,y4],[x1,y1]]
        """
        return [[bbox[0],bbox[1]],
                 [bbox[2],bbox[1]],
                 [bbox[2],bbox[3]],
                 [bbox[0],bbox[3]],
                 [bbox[0],bbox[1]]]
    
    aoi_floodTrace_bounds = tuple(generate_polygon(aoi_floodTrace['geometry'].total_bounds))
    
    Area_of_interestFloodTrace_geojson = {'type': 'Polygon',
                                          'coordinates': ((aoi_floodTrace_bounds),)}
    
    Pol_floodTrace = ee.Geometry(Area_of_interestFloodTrace_geojson)
    
    PolFloodTrace_2_clip = ee.Geometry.Polygon(aoi_floodTrace_bounds)
    
    Permanent_water_img = ee_download.permanent_water_image(date_event.year, PolFloodTrace_2_clip)
    
    # Adding the intersecions and water traces to the map
    fc = geemap.geopandas_to_ee(intersection)
    fc2 = geemap.geopandas_to_ee(aoi_floodTrace)
    fc3 = geemap.geopandas_to_ee(aoi_preFlood)
    fc4 = geemap.geopandas_to_ee(aoi_postFlood)
    

    Map = geemap.Map()

    visualization = {"bands": ['waterClass'],
                     "min": 0.0,
                     "max": 3.0,
                     "palette": ['cccccc', 'ffffff', '99d9ea', '0000ff']}

    Map.addLayer(Permanent_water_img, visualization, name="JRC Permanent water")

    Map.addLayer(Pol_floodTrace, {"color": 'FF000000'}, "Flood Trace Bounds")
    Map.addLayer(fc, {"color": 'FF0000'}, "Intersection")
    Map.addLayer(fc2, {"color": '000099'}, "Flood Trace")
    Map.addLayer(fc3, {"color": '009900'}, "Water Pre Flood")
    Map.addLayer(fc4, {"color": '000099'}, "Water Post Flood")



    # Map.centerObject(Pol_floodTrace, zoom=11)
    Map.centerObject(fc, zoom=11)

    folium.LayerControl(collapsed=False).add_to(Map)
    Map


    return intersection, Map

# Example

In [48]:
aoi_footprint_path = 'PACE/data/syd_reduced_footprint.geojson'
aoi_map_path = 'PACE/data/syd_reduced_map.geojson'

**Issue**: can't get geemap to display when calling the above function

In [78]:
intersection = get_intersection(aoi_footprint_path, aoi_map_path)

There are 399 buildings affected in this AOI


In [75]:
intersection.head()

,geometry,flood_cover,index
14713,"POLYGON ((150.80703 -32.48325, 150.80711 -32.4...",0.283155,14713
20010,"POLYGON ((150.80881 -32.29166, 150.80882 -32.2...",1.000000,20010
14697,"POLYGON ((150.80611 -32.29109, 150.80614 -32.2...",0.023421,14697
10710,"POLYGON ((150.80792 -32.28899, 150.80807 -32.2...",0.166624,10710
14729,"POLYGON ((150.80753 -32.28908, 150.80757 -32.2...",1.000000,14729


Wanted to try including both visualisations in the above function, but could not get it to work. 

In [67]:
intersection.index

Int64Index([14713, 20010, 14697, 10710, 14729,  4324, 12014,  9796,  4422,
            10005,
            ...
             7940,  8042,  9187,  9544, 13153, 14056, 15836, 18137, 19351,
            19932],
           dtype='int64', length=399)

In [71]:
intersection.explore(tooltip = ['index', 'flood_cover'], column= 'flood_cover', popup= True, style_kwds= dict(color= 'black'))